In [1]:
from aeon.datasets import load_classification
from sklearn.preprocessing import StandardScaler, LabelEncoder
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
import time

from ConvBN1d import ConvBN

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

dataset_name = 'WalkingSittingStanding' # Or 'Epilepsy' Or 'FordA'
# Load dataset
X_train, y_train, metadata = load_classification(dataset_name, return_metadata=True, split='train')
X_test, y_test = load_classification(dataset_name, split='test')
print(X_train.shape, X_test.shape)
if X_train.shape[0] < 200:
    train_size = int((X_train.shape[0] + X_test.shape[0]) * 3/4)
    x, y = load_classification(dataset_name)
    X_train, y_train = x[:train_size, :], y[:train_size]
    X_test, y_test = x[train_size:, :], y[train_size:]

# Flatten X if shape is (n_samples, 1, series_length) → (n_samples, series_length)
input_channels = 1
if X_train.ndim == 3:
    input_channels = X_train.shape[1]
    X_train = np.squeeze(X_train, axis=1) if input_channels == 1 else X_train
    X_test = np.squeeze(X_test, axis=1) if input_channels == 1 else X_test

seq_length = X_train.shape[-1]  # series length

# Encode labels
if y_train.dtype == object or isinstance(y_train[0], str):
    le = LabelEncoder()
    y_train = le.fit_transform(y_train)
    y_test = le.transform(y_test)

# Standard scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.reshape(-1, seq_length))
X_test_scaled = scaler.transform(X_test.reshape(-1, seq_length))

(7352, 3, 206) (2947, 3, 206)


In [3]:
X_train.shape, X_test.shape

((7352, 3, 206), (2947, 3, 206))

In [4]:
# ---- CNN ----
if input_channels == 1:
    X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32).unsqueeze(1).to(device)
    X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32).unsqueeze(1).to(device)
else:
    # Multichannel → keep original channels
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)

y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

num_classes = len(np.unique(y_train))

In [5]:
class LReLU(nn.Module):
    def __init__(self):
        super(LReLU, self).__init__()
        self.alpha = nn.Parameter(torch.tensor(5.0)) 
    def forward(self, x):
        return torch.nn.functional.relu(self.alpha*x)

In [6]:
class Network(nn.Module):
    def __init__(self, inpu_channels, seq_length, num_classes):
        super(Network, self).__init__()

        self.conv1_out = 64
        self.conv1_size = 11
        self.conv1_padding = 5


        self.conv2_out = 48
        self.conv2_size = 7
        self.conv2_padding = 3

        self.conv3_out = 32
        self.conv3_size = 3
        self.conv3_padding = 1

        self.fc1_out = num_classes

        self.q = 1e-6
        self.bias_trick_par = nn.Parameter(torch.tensor(0.00005))

        # First Convolutional Block

        self.block1 = ConvBN(in_channels=input_channels, out_channels=self.conv1_out, kernel_size=self.conv1_size, padding=self.conv1_padding, std = .05, bias_par_init=0.0015)
        self.block2 = ConvBN(in_channels=self.conv1_out, out_channels=self.conv2_out, kernel_size=self.conv2_size, padding=self.conv2_padding, std = .15, bias_par_init=0.0015)
               
        
        # torch.manual_seed(0)
        self.w2 = nn.Parameter(torch.randn(self.conv2_out * (seq_length// 2 // 2), self.fc1_out))
        nn.init.normal_(self.w2, mean=0.0, std=.6)

        self.dropout = nn.Dropout(0.5)

        self.relu = LReLU()

        self.pool = nn.MaxPool1d(2)




    def forward(self, x):
        x = self.pool(self.relu(self.block1(x)))
        x = self.pool(self.relu(self.block2(x)))
        
        x = x.view(x.size(0), -1)
    
        x = x + self.bias_trick_par
        x_norm = x / (x.norm(p=2, dim=1, keepdim=True) + self.q)  # Normalize input x
        w2_norm = self.w2 / (self.w2.norm(p=2, dim=1, keepdim=True) + self.q)  # Normalize weights
        x = torch.matmul(x_norm, w2_norm) # Matrix multiplication 

        # Return raw logits (no softmax here, CrossEntropyLoss handles it)
        return x

    def custom_round(self, n):
        remainder = n % 1000
        base = n - remainder
        if remainder >= 101:
            return base + 1000
        elif remainder <= 100:
            return base

    @torch.no_grad()             # no autograd graph
    def flip_sign_(self, tensor: torch.Tensor, percentage: float) -> torch.Tensor:
        """
        Flip the sign of a random subset of elements *in place*.
    
        Args:
            tensor (torch.Tensor): Any shape, modified in place.
            percentage (float): 0‒1 fraction of elements to flip.
    
        Returns:
            torch.Tensor: The same tensor object (for chaining).
        """
        if percentage <= 0.0:
            return tensor
        if percentage >= 1.0:
            tensor.mul_(-1)
            return tensor                    # all elements flipped
    
        numel = tensor.numel()
        num_to_flip = int(numel * percentage)
        if num_to_flip == 0:
            return tensor
    
        flat = tensor.view(-1)               # view ↔ no copy
        idx = torch.randint(0, numel, (num_to_flip,),
                            device=flat.device)
        flat[idx] *= -1                      # in-place sign change
        return tensor

    def init_hdc(self, ratio, seed, flip_perc=None):
        if not isinstance(ratio, (tuple, int)):
            raise TypeError("ratio must be a tuple of size 4 or and integer")

        elif isinstance(ratio, (int)):
            ratio = (ratio, ratio, ratio, ratio)
            
        if not isinstance(seed, (tuple)):
            raise TypeError("seed must be a tuple of size 4")
        
        self.block1.init_hdc(ratio = ratio[0], seed = seed[0], flip_perc=flip_perc)
        self.block2.init_hdc(ratio = ratio[1], seed = seed[1], flip_perc=flip_perc)
                
        self.n_last = self.w2.size(0)
        self.nHDC_last = int(self.custom_round(ratio[2] * self.n_last)) if ratio[2]<1000 else int(ratio[2])
        torch.manual_seed(seed[2])
        self.g = (torch.randn(self.w2.size(0), self.nHDC_last, device=self.w2.device)).to(torch.half)
        self.wg = torch.sign(torch.matmul(self.g.t(), self.w2.to(torch.half)))

        if flip_perc is not None and flip_perc > 0.0:
            self.flip_sign_(self.wg, flip_perc)


    def hdc(self, x):
        x = self.pool(self.relu(self.block1.hdc(x)))
        x = self.pool(self.relu(self.block2.hdc(x)))

        x = x.view(x.size(0), -1)
        
        x = x + self.bias_trick_par
        x = torch.sign(torch.matmul(x.to(torch.half), self.g))

        return x
        
    def classification_layer(self, x):
        x = x @ self.wg
        return x

In [7]:
from tqdm import tqdm
from torch.utils.data import Subset
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


torch.cuda.empty_cache()
model = Network(input_channels, seq_length, num_classes).to(device)
avg_test_acc = torch.load('avg_test_acc.pt', weights_only=True)
model.load_state_dict(torch.load(f'{dataset_name}_GNet_Training_{avg_test_acc:.4f}.pth', weights_only = True))

<All keys matched successfully>

In [8]:
model.to(torch.half).to(device)
model.eval()
n_splits = 20
split_size = len(test_dataset) // n_splits # 10000 // 20 = 500scales = np.arange(0.2, 1.21, 0.1)
print(split_size)
scales = np.arange(0.0, 0.51, 0.05)
hyperdim = 15_000
print(len(scales))
test_loader = DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=0, pin_memory=False)
NHDC = np.zeros((len(scales), 3))
accuracies = np.zeros((len(scales), n_splits)) 
num_workers = 0
pin_memory = False
for i, perc in enumerate(scales):
    # print(ratio)
    indices = list(range(len(test_dataset)))
    np.random.seed(42)
    np.random.shuffle(indices)  # or random.shuffle(indices)
    for split_idx in tqdm(range(n_splits)):
        start_idx = split_idx * split_size
        end_idx = start_idx + split_size
        split_indices = indices[start_idx:end_idx]
        split_subset = Subset(test_dataset, split_indices)
        split_loader = torch.utils.data.DataLoader(split_subset, batch_size=20, shuffle=False,
                                                   num_workers=num_workers, pin_memory=pin_memory)
        torch.manual_seed(split_idx+4)
        random_seeds = tuple(torch.randint(0, 1000, (1,)).item() for _ in range(4))
        torch.cuda.empty_cache()
        
        model.init_hdc(hyperdim, random_seeds, perc)
        # model.init_hdc(scale, random_seeds)
        correct = 0
        total = 0
        
    
        with torch.no_grad():
            for images, labels in (split_loader):
                images, labels = images.cuda(non_blocking=True), labels.cuda(non_blocking=True)
                output = model.hdc(images.to(torch.half))
                output = model.classification_layer(output.to(torch.half))
                _, predicted = torch.max(output.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
    
        acc = 100 * correct / total
    
        accuracies[i, split_idx] = acc
    
        # print(f"Split {split_idx+1}: accuracy = {acc:.2f}%, time = {dt:.2f} sec")
    print(f'Block1: {model.block1.nHDC}, Block2: {model.block2.nHDC}, Classification Layer: {model.nHDC_last}, Accuracy: {np.mean(accuracies[i]):.2f}%')


147
11


100%|██████████| 20/20 [00:20<00:00,  1.01s/it]


Block1: 15000, Block2: 15000, Classification Layer: 15000, Accuracy: 85.92%


100%|██████████| 20/20 [00:19<00:00,  1.00it/s]


Block1: 15000, Block2: 15000, Classification Layer: 15000, Accuracy: 84.83%


...

Block1: 15000, Block2: 15000, Classification Layer: 15000, Accuracy: 42.79%


100%|██████████| 20/20 [00:20<00:00,  1.00s/it]


Block1: 15000, Block2: 15000, Classification Layer: 15000, Accuracy: 39.25%


100%|██████████| 20/20 [00:19<00:00,  1.00it/s]

Block1: 15000, Block2: 15000, Classification Layer: 15000, Accuracy: 36.87%


In [9]:
np.mean(accuracies, axis=1)

array([85.91836735, 84.82993197, 83.23129252, 80.91836735, 77.58503401,
       69.45578231, 56.19047619, 47.58503401, 42.78911565, 39.25170068,
       36.8707483 ])

In [10]:
from scipy.io import savemat
savemat(f'{dataset_name}_HDCGNet.mat', {f'{dataset_name}_HDCGNet': accuracies})

In [11]:
np.std(accuracies, axis=1)

array([2.61794305, 2.6002061 , 3.63977365, 4.08998578, 5.47809153,
       6.49615342, 6.92208209, 4.97796989, 5.29957733, 5.63066448,
       6.8183494 ])